In [2]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
from pathlib import Path

import pandas as pd
from loguru import logger

from plex_pipe.core_cutting.channel_scanner import discover_channels
from plex_pipe.core_cutting.controller import CorePreparationController
from plex_pipe.core_cutting.file_io import LocalFileStrategy
from plex_pipe.utils.config_loaders import load_analysis_settings

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Load analysis settings

In [3]:
# load analysis configuration
settings_path = r'C:\sdata_toy_example/analysis_settings_LCB004-N-P_todel.yaml'

settings = load_analysis_settings(settings_path)
settings

2025-11-19 23:30:51.978 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:51 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


AnalysisConfig(general=GeneralSettings(image_dir='/mnt/r/CellDive/LCB004-N-P/LCB004-N-P_Final', analysis_name='example_analysis', local_analysis_dir='/mnt/c/sdata_toy_example', remote_analysis_dir='/ix1/kkedziora', log_dir=None), core_detection=CoreDetectionSettings(detection_image='LCB004-N-P_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=10000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=None, use_markers=['DAPI', 'ECad'], ignore_markers=['Antibody1'], margin=0, mask_value=0, transfer_cleanup_enabled=False, core_cleanup_enabled=False), additional_elements=[NormalizeStep(category='image_filter', type='normalize', input='DAPI', output='DAPI_norm', keep=False, parameters=Params(low=1.0, high=99.8)), NormalizeStep(category='image_filter', type='normalize', input='ECad', output='ECad_norm', keep=False, parameters=Params(lo

### Define the logger

In [6]:
log_file = settings.log_dir_path / f"cores_cutting_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

### Define cores for processing

In [7]:
df_path = settings.core_info_file_path.with_suffix('.pkl')

df = pd.read_pickle(df_path)
df.head()

,roi_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,Core_000,14208.0,14784.0,70656.0,71296.0,rectangle,"[[14208.0, 71296.0], [14784.0, 71296.0], [1478..."


In [ ]:
# # create a subset of cores (optional)
# df = df[:1]
# df

,roi_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,Core_000,1984.0,49408.0,192.0,39296.0,polygon,"[[11328.0, 15296.0], [6336.0, 16064.0], [2624...."


## Local files

In [8]:
channel_map = discover_channels(Path(settings.general.image_dir),
                                include_channels=settings.core_cutting.include_channels,
                                exclude_channels=settings.core_cutting.exclude_channels,
                                use_markers=settings.core_cutting.use_markers,
                                ignore_markers=settings.core_cutting.ignore_markers)

2025-11-18 14:01:28.557 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:76 - Discovered 19 channels:
2025-11-18 14:01:28.559 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:78 - 001_AP2B <- R:\CellDive\LCB004-N-P\LCB004-N-P_Final\LCB004-N-P_1.0.4_R000_Cy5_AP2B-AF647_FINAL_AFR_F.ome.tif
2025-11-18 14:01:28.559 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:78 - 001_DAPI <- R:\CellDive\LCB004-N-P\LCB004-N-P_Final\LCB004-N-P_1.0.4_R000_DAPI__FINAL_F.ome.tif
2025-11-18 14:01:28.560 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:78 - 001_FOXA1 <- R:\CellDive\LCB004-N-P\LCB004-N-P_Final\LCB004-N-P_1.0.4_R000_Cy3_FOXA1-AF555_FINAL_AFR_F.ome.tif
2025-11-18 14:01:28.561 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:78 - 002_DAPI <- R:\CellDive\LCB004-N-P\LCB004-N-P_Final\LCB004-N-P_2.0.4_R000_DAPI__FINAL_F.ome.tif
2025-11-18 14:01:28.561 | INFO     | pl

In [ ]:
# # for tests you can request a small set of channels here
# # then send 'short_map' to the controller instead of 'channel_map'
# selected_keys = ["DAPI", "CD3"]
# short_map = {k: channel_map[k] for k in selected_keys if k in channel_map}
# short_map

{'DAPI': 'R:\\CellDive\\BLCA-1B\\BLCA-1B_Final\\BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'CD3': 'R:\\CellDive\\BLCA-1B\\BLCA-1B_Final\\BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif'}

In [9]:
strategy = LocalFileStrategy()

controller = CorePreparationController(
    metadata_df = df, # df defines which cores to process
    image_paths = channel_map, # defines which channels to use
    temp_dir = settings.cores_dir_tif_path,
    output_dir = settings.cores_dir_output_path,
    file_strategy = strategy,
    margin = settings.core_cutting.margin,
    mask_value = settings.core_cutting.mask_value,
    max_pyramid_levels = settings.sdata_storage.max_pyramid_level,
    chunk_size = settings.sdata_storage.chunk_size,
    downscale = settings.sdata_storage.chunk_size,
    core_cleanup_enabled = False,
    )


controller.run()

2025-11-18 14:01:32.214 | INFO     | plex_pipe.core_cutting.controller:run:117 - Starting controller run loop...
2025-11-18 14:01:32.223 | INFO     | plex_pipe.core_cutting.controller:run:127 - Channel AP2B file available at R:\CellDive\LCB004-N-P\LCB004-N-P_Final\LCB004-N-P_1.0.4_R000_Cy5_AP2B-AF647_FINAL_AFR_F.ome.tif.
2025-11-18 14:01:33.078 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:93 - Cut and saved core Core_000, channel AP2B.
2025-11-18 14:01:33.082 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:100 - Closed file handle for channel AP2B.
2025-11-18 14:01:33.087 | INFO     | plex_pipe.core_cutting.controller:run:127 - Channel DAPI file available at R:\CellDive\LCB004-N-P\LCB004-N-P_Final\LCB004-N-P_1.0.4_R000_DAPI__FINAL_F.ome.tif.
2025-11-18 14:01:33.328 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:93 - Cut and saved core Core_000, channel DAPI.
2025-11-18 14:01:33.333 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:100 - Clo